## Importação das bibliotecas

In [232]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go
import openpyxl
import os

## Sem esse comando os graficos não sao exibidos no notebook

In [233]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [234]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm = int(input('Digite um valor númerico para a média móvel'))

# Recebe do usuário o valor das médias 
qtt_std = int(input('Digite a quantidade de desvio padrão da estratégia'))

# Define o lote padrão de 100 ações
capital = int(input('Digite o capital disponível para essa estratégia'))

# Define o periodo de busca do historico de preços
ini_date = datetime.date(2013,1,1)
end_date = datetime.date(2020,12,31)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date, interval='1d')
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,12.12,12.70,12.12,12.62,8.314431,7098900
1,2013-01-03,12.64,12.89,12.37,12.80,8.433021,4449600
2,2013-01-04,12.74,12.78,12.16,12.32,8.116780,8732900
3,2013-01-07,12.37,12.49,12.04,12.26,8.077252,4316000
4,2013-01-08,12.38,12.71,12.10,12.15,8.004782,6697500


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [235]:
df_stock_prices['MM'] = df_stock_prices.Close.rolling(window=num_mm).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices['Close'] - df_stock_prices['MM']

df_stock_prices['Start'] = 0.00
df_stock_prices['Stop'] = 0.00
df_stock_prices['Profit'] = 0.00

df_stock_prices['MM'] = df_stock_prices['MM'].fillna(0)

# Excluindo do Data frame os registros com a média móvel zerada pois não dá pra fazer backtest nesses registros
df_stock_prices = df_stock_prices.loc[df_stock_prices['MM']!=0]
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM,SPREAD_MM,Start,Stop,Profit
199,2013-10-17,11.45,11.72,11.18,11.57,8.109100,12738500,8.63500,2.93500,0.0,0.0,0.0
200,2013-10-18,11.67,11.85,11.46,11.76,8.242268,7737500,8.63070,3.12930,0.0,0.0,0.0
201,2013-10-21,11.67,11.99,11.63,11.91,8.347398,5557700,8.62625,3.28375,0.0,0.0,0.0
202,2013-10-22,11.88,12.42,11.83,12.40,8.690826,8810000,8.62665,3.77335,0.0,0.0,0.0
203,2013-10-23,12.24,12.40,12.15,12.21,8.557660,8196400,8.62640,3.58360,0.0,0.0,0.0
204,2013-10-24,12.25,12.33,11.81,11.88,8.326372,8850300,8.62505,3.25495,0.0,0.0,0.0
205,2013-10-25,11.92,11.92,11.56,11.70,8.200215,6823100,8.62110,3.07890,0.0,0.0,0.0
206,2013-10-28,11.72,12.07,11.69,11.94,8.368424,5781400,8.61725,3.32275,0.0,0.0,0.0
207,2013-10-29,12.01,12.07,11.73,11.82,8.284319,3825800,8.61325,3.20675,0.0,0.0,0.0
208,2013-10-30,11.93,12.14,11.86,11.95,8.375434,5971500,8.60950,3.34050,0.0,0.0,0.0


## Plota o gráfico com o preço de fechamento e as médias móveis

In [236]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

close_prices2 = go.Candlestick(
                                x = df_stock_prices['Date'],
                                open = df_stock_prices['Open'],
                                high = df_stock_prices['High'],
                                low = df_stock_prices['Low'],
                                close = df_stock_prices['Close'])

mm = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM'],
                    name = 'Média Móvel',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 

In [237]:
status = False
start_price = 0
last_spread = 0
price_std = (df_stock_prices['SPREAD_MM'].std() *-1) * 0.5
price_value_std = price_std * qtt_std

print(price_std)
print(price_value_std)
for row in df_stock_prices.itertuples():
    lote = capital / row.Close
    #print(lote)
    if (row.SPREAD_MM < price_value_std) & (status == False):
        print(row.Index)
        status = True
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
    elif (row.SPREAD_MM < (price_value_std)) & (status == True):
        print(status)
        df_stock_prices.at[row.Index, 'Start'] = start_price
        status = True
    elif (row.SPREAD_MM < price_std) & (row.SPREAD_MM > price_value_std) & (status == True):
        df_stock_prices.at[row.Index, 'Start'] = start_price
        status = True
    elif (row.SPREAD_MM > price_std) & (status == True):
        print(start_price)
        df_stock_prices.at[row.Index, 'Profit'] = (row.Close - start_price) * lote
        df_stock_prices.at[row.Index, 'Stop'] = row.Close
        status = False
        start_price = 0


df_stock_prices.head(285)

-1.289295547355881
-5.157182189423524
1781
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
8.300000190734863


,Date,Open,High,Low,Close,Adj Close,Volume,MM,SPREAD_MM,Start,Stop,Profit
199,2013-10-17,11.45,11.72,11.18,11.57,8.109100,12738500,8.63500,2.93500,0.0,0.0,0.0
200,2013-10-18,11.67,11.85,11.46,11.76,8.242268,7737500,8.63070,3.12930,0.0,0.0,0.0
201,2013-10-21,11.67,11.99,11.63,11.91,8.347398,5557700,8.62625,3.28375,0.0,0.0,0.0
202,2013-10-22,11.88,12.42,11.83,12.40,8.690826,8810000,8.62665,3.77335,0.0,0.0,0.0
203,2013-10-23,12.24,12.40,12.15,12.21,8.557660,8196400,8.62640,3.58360,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
479,2014-12-04,5.74,5.74,5.44,5.54,4.260572,4078900,9.38060,-3.84060,0.0,0.0,0.0
480,2014-12-05,5.55,5.75,5.47,5.54,4.260572,13551100,9.35195,-3.81195,0.0,0.0,0.0
481,2014-12-08,5.53,5.62,5.22,5.22,4.014474,5408800,9.32450,-4.10450,0.0,0.0,0.0
482,2014-12-09,5.21,5.25,5.02,5.07,3.899115,4878000,9.29355,-4.22355,0.0,0.0,0.0


In [238]:
# No periodo diario , MM de 50 periodos no gráfico diário e 3 desvio padrão a estratégia está sendo lucrativa
print('O resultado da estratégia é R$ %7.2f' % df_stock_prices['Profit'].sum())

O resultado da estratégia é R$ 1139.53


In [281]:
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']!=0].count()
entry_count = df_stock_prices_count['Profit']
print('O número de entradas executadas foi de %7.0f' % entry_count)

O número de entradas executadas foi de       1


In [287]:
import os
# Se o arquivo existe ele será excluido para ser criado novamente
if os.path.exists('spread_mm.xlsx'):
  os.remove('spread_mm.xlsx')
else:
  print("The file does not exist")

# Exporta o data frame para um arquivo no Excel
df_stock_prices.to_excel('spread_mm.xlsx', sheet_name='Spread_MM', index=False)

# Cria um resumo do backtest em uma nova guia do Excel
wb = load_workbook('spread_mm.xlsx')

sheet = wb.create_sheet(index = 0 , title = "Resumo_Backtest") 

sheet['A1'] = 'Resumo do Backtest'
sheet['A2'] = 'Papel:'
sheet['B2'] = stock.upper()
sheet['A3'] = 'Capital:'
sheet['B3'] = capital
sheet['B3'].number_format
sheet['A4'] = 'Média Móvel Utilizada: '
sheet['B4'] = num_mm
sheet['A5'] = 'Resultado Financeiro: R$'
sheet['B5'] = df_stock_prices['Profit'].sum()
sheet['A6'] = 'Número de entradas Executadas: '
sheet['B6'] = entry_count
wb.save('spread_mm.xlsx')